<a href="https://colab.research.google.com/github/rarodrigo-datasc/emenda-parlamentar-spark/blob/main/emenda_parlamentar_spark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1. Preparação do Ambiente

### 1.1 Instalar as dependências para **Spark**

* openjdk-8-jdk-headless: biblioteca `java` sem interface gráfica usada pelo Spark
* findspark: a biblioteca Python `findspark` facilita a integração do Spark com o ambiente Python.

In [1]:
!apt update -qq
!apt upgrade -y
!apt install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.1.2/spark-3.1.2-bin-hadoop2.7.tgz
!tar xf spark-3.1.2-bin-hadoop2.7.tgz
!pip install -q findspark

47 packages can be upgraded. Run 'apt list --upgradable' to see them.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
Calculating upgrade... Done
The following packages have been kept back:
  libcudnn8 libcudnn8-dev libnccl-dev libnccl2
The following packages will be upgraded:
  base-files bash binutils binutils-common binutils-x86-64-linux-gnu bsdutils coreutils
  cuda-compat-12-2 cuda-keyring cuda-toolkit-12-config-common cuda-toolkit-config-common dpkg
  dpkg-dev libbinutils libblkid1 libc-bin libcdio19 libctf-nobfd0 libctf0 libcups2 libdpkg-perl
  libgnutls30 libldap-2.5-0 libmount1 libpam-modules libpam-modules-bin libpam-runtime libpam0g
  libperl5.34 libprocps8 libsmartcols1 libudev1 linux-libc-dev login mount openssl passwd perl
  perl-base perl-modules-5.34 procps tar util-linux
43 upgraded, 0 newly installed, 0 to remove and 4 not upgraded.
Need to get 59.9 MB of archives.
After this operation, 124 kB of additional disk space w

### 1.2 Configurar as variáveis de ambiente

In [2]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop2.7"

### 1.3 Inicializar o módulo `findspark`

Configura o ambiente Python para encontrar os módulos e bibliotecas do Spark.

In [3]:
import findspark
findspark.init('spark-3.1.2-bin-hadoop2.7')


## 2. Preparação dos Dados

### 2.1 Configurar SparkSession

* Criar uma sessão spark para interagir com os dados usando DataFrames e Datasets;


In [4]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local[*]').appName("emenda-parlamentar-spark").config('spark.ui.port', '4050').getOrCreate()


### 2.2 Carregar os dados

Procedimento manual

* Fazer download da base de dados em `https://github.com/rarodrigo-datasc/emenda-parlamentar-spark/blob/9ad7c0777998d337f00ae00ee11efedc7aa54127/EmendasParlamentares.zip`;
* Fazer upload da base de dados para o projeto;

Extração da base

* Extrair a base de dados no formato ".zip";
* Ler os dados extraídos.

In [6]:
import zipfile
zipfile.ZipFile('./EmendasParlamentares.zip').extractall()


In [14]:
df = spark.read.csv('./Emendas.csv', sep=';', encoding='ISO-8859-1', header=True)
df.printSchema()
df.show()


root
 |-- Código da Emenda: string (nullable = true)
 |-- Ano da Emenda: string (nullable = true)
 |-- Tipo de Emenda: string (nullable = true)
 |-- Código do Autor da Emenda: string (nullable = true)
 |-- Nome do Autor da Emenda: string (nullable = true)
 |-- Número da emenda: string (nullable = true)
 |-- Localidade do gasto: string (nullable = true)
 |-- Código Função: string (nullable = true)
 |-- Nome Função: string (nullable = true)
 |-- Código Subfunção: string (nullable = true)
 |-- Nome Subfunção: string (nullable = true)
 |-- Valor Empenhado: string (nullable = true)
 |-- Valor Liquidado: string (nullable = true)
 |-- Valor Pago: string (nullable = true)
 |-- Valor Restos A Pagar Inscritos: string (nullable = true)
 |-- Valor Restos A Pagar Cancelados: string (nullable = true)
 |-- Valor Restos A Pagar Pagos: string (nullable = true)

+----------------+-------------+--------------------+-------------------------+-----------------------+----------------+--------------------+--